<div style="text-align: center;">
<h1>Assignment 2: Stream Ciphers</font></h1>
<h2>Course: Elements of Applied Data Security</font></h2>

<center><img src="../images/unibo.png" alt="unibo_logo" width="200"/></center>

<h3>Professor: Alex Marchioni and Livia Manovi</font></h3>
<h3>University: Università degli Studi di Bologna</font></h3>
<h3>Author: Lluis Barca Pons</font></h3>
<h3>Date: 2024-04-23</font></h3>
</div>

Stream cipher is an encryption method that processes the plaintext by encrypting one bit or byte at a time with a pseudorandom keystream. This type of cipher operates by generating a stream of cryptographic keys that are then bitwise XORed with the plaintext bits to produce ciphertext. Stream ciphers are noted for their speed and simplicity, making them ideal for applications that require fast, continuous data encryption such as real-time communication. They offer advantages in scenarios where quick encryption and decryption are needed without the added complexity and size constraints posed by block ciphers.

# Importing libraries and modules

In [1]:
from streamcipher import LFSR
from streamcipher import berlekamp_massey
from streamcipher import ShrinkingGenerator, SelfShrinkingGenerator

from itertools import islice

## LFSR

LFSR (Linear Feedback Shift Register) is a simple yet powerful hardware-efficient structure used to generate pseudorandom binary sequences. It consists of a series of flip-flops (registers), each capable of storing one bit of data, shifted in steps under the control of a clock. At each step, a feedback function, typically a linear combination of bits in specific positions of the register (taps), is computed using the XOR operation. This result is then fed back as input to the register. The configuration of taps determines the characteristics of the output sequence, such as its period and randomness. LFSRs are widely used in cryptography, digital signal processing, and error detection protocols.

### Testing

Test LFSR implementation (all attributes and mehtods) with the polynomial `poly = [4, 1, 0]`.

In [2]:
def lfsr_generator(poly_list, state_list=None, bm=False):
    """ LFSR generator function
    This function generates the LFSR sequence and optionally applies the Berlekamp-Massey algorithm to find the polynomial that generates the sequence.

    Args:
        poly_list: list of integers representing the polynomial coefficients
        state_list: list of integers representing the initial state of the LFSR
        bm: boolean flag to apply the Berlekamp-Massey algorithm. Default is False
    """

    lfsr = LFSR(poly_list, state_list)

    if not bm :
        print('\nstate     b fb')
        for _ in islice(lfsr, 10):
            print(lfsr)

    if bm :
        full_cycle = lfsr.cycle()
        print(f'\nfull LFSR cycle -> {full_cycle}')

        P = berlekamp_massey(full_cycle)

        print(f'\nBerlekamp-Massey algorithm -> {P}')
        P.reverse()
        nonzero = [i for i, e in enumerate(P) if e != 0]
        poly_list = [f'x^{d}' for d in nonzero if d!= 0]
        poly_list.reverse()
        poly = "+".join(poly_list) + "+1"
        print(poly)

In [3]:
lfsr_generator([4, 1, 0])

Initial state: [1, 1, 1, 1]

state     b fb
[0, 1, 1, 1] 1 0
[0, 0, 1, 1] 1 0
[0, 0, 0, 1] 1 1
[1, 0, 0, 0] 0 0
[0, 1, 0, 0] 0 0
[0, 0, 1, 0] 0 1
[1, 0, 0, 1] 1 1
[1, 1, 0, 0] 0 0
[0, 1, 1, 0] 0 1
[1, 0, 1, 1] 1 0


## Berlekamp Massey

The Berlekamp-Massey algorithm is an efficient method used to determine the minimal polynomial of a linear feedback shift register (LFSR) from a given sequence of binary data. This polynomial represents the shortest LFSR that can generate the observed sequence. The algorithm systematically adjusts a simulated LFSR's taps through discrepancy calculations, effectively minimizing the polynomial's length while ensuring it aligns with the input sequence. It is widely used in error-correcting codes to optimize decoding processes and in cryptography to analyze the structure of pseudorandom number generators, enabling the reconstruction of the original LFSR configuration from output sequences.

### Testing

Test the implementation Berlekamp-Massy Algorithm with the sequence produced by the following LFSR:
- `poly = [3, 1, 0]` and `state = '\x06'`
- `poly = [5, 2, 1, 0]` and `state = '\x05'`
- `poly = [96, 81, 17, 15, 0]` and `state = b'streamcipher'`

In [4]:
def state_to_bits(s):
    """ Convert a state to a list of bits

    Args:
        s: A binary string representing the state of a LFSR

    state_to_bitsReturns:
        bits: A list of bits representing the state
    """

    return int.from_bytes(s, byteorder='big')

In [5]:
poly_list = [3, 1, 0]
state = state_to_bits(b'\x06')
lfsr_generator(poly_list, state, bm=True)

Initial state: [1, 1, 0]

full LFSR cycle -> [1, 1, 1, 0, 0, 1, 0]

Berlekamp-Massey algorithm -> [0, 3, 6]
x^1+1


In [6]:
poly_list = [5, 2, 0]
state = state_to_bits(b'\x05')
lfsr_generator(poly_list, state, bm=True)

Initial state: [0, 0, 1, 0, 1]

full LFSR cycle -> [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1]

Berlekamp-Massey algorithm -> [0, 2, 4, 6, 8, 9, 17, 18, 19, 22]
x^8+x^7+x^6+x^5+x^4+x^3+x^2+x^1+1


In [7]:
poly_list = [96, 81, 17, 15, 0]
state = state_to_bits(b'streamcipher')
lfsr_generator(poly_list, state, bm=True)

Load `'binary_sequence'` and find the polynomial corresponding to the shortest LFSR that can produce that sequence.

In [8]:
# Loading the binary_sequence.bin file
with open('binary_sequence.bin', 'rb') as f:
    data = f.read()

# Converting the binary data to bits
bits_data = []
for byte in data:
    for i in range(8): # 8 bits in a byte
        bits_data.append((byte >> i) & 1)

# Finding the exponents of the polynomial
poly = [index for index, bit in enumerate(bits_data) if bit == 1]

# Finding the polynomial corresponding to the shortest LFSR that generates the sequence
lfsr_generator(poly_list=poly, bm=True)

## LFSR-based Generator (Shrinking or Self-Shrinking Generator)

### Testing

Test Generator implementation (all attributes and methods).

In [9]:
# Testing the Shrinking Generator
print("\nShrinking Generator")
poly_list1 = [3, 2, 0]
poly_list2 = [3, 1, 0]
state1 = int.from_bytes(b'\x06', byteorder='big')
state2 = int.from_bytes(b'\x05', byteorder='big')

sg = ShrinkingGenerator(poly_list1, poly_list2, state1, state2)

# Test all attributes
print(f'LFSRA: {sg.lfsrA}')
print(f'LFSRS: {sg.lfsrS}')
print(f'output: {sg.output}')

# Testing the next method
print(next(sg))
print(next(sg))
print(next(sg))

# Testing the iter method
iterator = iter(sg)
print(next(iterator))
print(next(iterator))
print(next(iterator))


Shrinking Generator
Initial state: [1, 1, 0]
Initial state: [1, 0, 1]
LFSRA: [1, 1, 0] [1] 1
LFSRS: [1, 0, 1] [1] 1
output: None
0
1
1
0
0
0


In [10]:
# Testing the Self Shrinking Generator
print("\nSelf Shrinking Generator")
poly_list = [3, 2, 0]
sbit = 4
state = int.from_bytes(b'\x06', byteorder='big')

ssg = SelfShrinkingGenerator(poly_list, sbit, state)

# Test all attributes
print(f'LFSR: {ssg.lfsr}')
print(f'sbit: {ssg.sbit}')
print(f'output: {ssg.output}')

# Testing the next method
print(next(ssg))
print(next(ssg))
print(next(ssg))

# Testing the iter method
iterator = iter(ssg)
print(next(iterator))
print(next(iterator))
print(next(iterator))


Self Shrinking Generator
Initial state: [1, 1, 0]
LFSR: [1, 1, 0] [1] 1
sbit: 4
output: None
0
0
0
0
0
0


Decrypt the ciphertext in `ciphertext_shrinking.bin` or `ciphertext_selfshrinking.bin`.

In [11]:
# Transform the name group into a bytes object (big endian)
group_name = b'EncryptionEagles'

# Iterate over the 8 bits of the byte and shift the byte i times to the right and check if the least significant bit is 1.
group_bits = [bool(byte & (1 << i)) for byte in group_name for i in range(8)]

# Comptute the parity bit
parity = sum(group_bits) % 2
print(f'Parity bit: {parity}')

Parity bit: 1


In [12]:
polyA = [16, 15, 12, 10, 0]
polyS = [24, 11, 5, 2, 0]
stateA = b'\xc5\xd7'
stateS = b'\x14\x84\xf8'

In [13]:
poly = [32, 16, 7, 2, 0]
selection_bit = 4
state = b'mJ\x9by'

In [14]:
def decrypt_file(file, sg):
    """ Decrypt a file using a stream cipher generator

    Args:
        file: file to decrypt
        sg: stream cipher generator

    Returns:
        plaintext: decrypted file
    """

    with open(file, 'rb') as f:
        ciphertext = f.read()

    plaintext = []
    for byte in ciphertext:
        keystream_byte = next(sg)
        plaintext_byte = byte ^ keystream_byte
        plaintext.append(plaintext_byte)

    print(f'\nPlaintext coded: {bytes(plaintext)}')

    return bytes(plaintext).decode('utf-8', errors='replace')

In [15]:
# Choosing the assigned generator depending on the parity bit
if parity == 0:
    sg = ShrinkingGenerator(polyA, polyS, state_to_bits(stateA), state_to_bits(stateS))
    plaintext = decrypt_file('./ciphertext_shrinking.bin', sg)
    print(plaintext)
else:
    ssg = SelfShrinkingGenerator(poly, selection_bit, state_to_bits(state))
    plaintext = decrypt_file('./ciphertext_selfshrinking.bin', ssg)
    print(f'\nPlaintext decoded: {plaintext}')

Initial state: [0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1]

Plaintext coded: b'\x03\x901-o\xcd\xa0M2\xe9\x01C^\x07jvs\xc3\xfe\x81p\xa9\xe7h/\xb8\xa5\xe1+y5F\xb0\xf7\x98V\x92\xb4\xeb=l\x13\x93\xf8\xde\xf8\xda\x19,\x97\xa7[,\t3a(\xd75j=\xb6\xafP\x01\xad\xbb\x86\x93\xa0\x89\x08\xef\xc3\xd2B\xea\'\xe2\xd3\x93\xec\xa5\x86\xedp\x1a\x11\x00\x0c\x0f\xdbY\xfe\xebM\xa8\xcbl\xdc\xc6_R\xf8\xb7\xf9\x8d\xf9~\xae\xb3\x10\xe6m\x13=\xe8\x84u\x99\x90X\xf3\x1dx=\xed\x92\xc5;\xdd\xc7g\x90\xd0u\t\xf2\x11\xef\x1a^\xff\xe0\x12z\x1e\xfa\x08\xb88\xf4~j\x95\xdd\x85\xa1\xac o\x00\xe6\xf0~\xb1\xd2\'n\x88\xc3\xa7\x0b3X\x00r\xda#\xf0bz\xd3Y\xaey\x7fiz\xdf\xa5\xd5\x9f\xba\x0b\xcc\x01l\xcd\xb6\xc0%/\xdfn\x9d\x7f\xe7,}Q\xe9\xd7\x8b\xab\xcc\xfb\xf9\xad\x86\xb3Y\x07\x7fQ[\x97\x0f%p}\xbe\x87C\xff\x17\xfd[_\xbf\x03\xe48\x9f\x1b\xd0\x1c8x\x96&eg8\xd1\\\xca\xe5!\xa4\x0b\x8c\xf5\x93m\x86\x9d\x87D\xbfk\x01\x07\x1fb\x9d\xf1\xd9\x9b\x8e\xaa}\x1f\xb6\x86\xad\xcc\xb6!\x8b\x02K\

## Conclusion

Stream Ciphers represent a fundamental category of symmetric key cipher where plaintext data is encrypted one bit at a time. This method contrasts with block ciphers, which process blocks of data in fixed sizes. The primary advantage of stream ciphers is their efficiency and simplicity, making them highly suitable for environments where data is transmitted continuously, such as in telecommunications and real-time applications.

In this project, I employed a specific type of stream cipher known as a Shrinking Generator, which leverages the properties of Linear Feedback Shift Registers (LFSRs) to produce a pseudorandom bit stream. This keystream is then used to encrypt or decrypt messages via the XOR operation, offering a blend of simplicity and speed alongside decent security under certain conditions. LFSRs are particularly noted for their efficient hardware implementation and predictable cycle length, which can be precisely controlled by choosing appropriate feedback taps.

Also, I used the Berlekamp-Massey algorithm that is crucial in analyzing LFSRs within cryptographic applications, as it efficiently determines the minimal polynomial of an LFSR based on its output sequence. This capability is instrumental in cryptanalysis, helping to expose vulnerabilities in the stream cipher if the LFSR sequence can be predicted or reconstructed.

Overall, the integration of LFSRs with stream ciphers, analyzed through methods like the Berlekamp-Massey algorithm, demonstrates a robust approach to achieving fast and secure data encryption. However, the security of such systems heavily relies on the proper selection of the LFSR parameters and the unpredictability of the output sequence, emphasizing the need for careful cryptographic design and analysis.